In [8]:
import os
import json
import pandas as pd
from random import randrange
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip
from tqdm import tqdm

In [9]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]

In [10]:
#input json
#set medical video download folder
medvid_folder = "/home/hlpark/shared/MedVidQA/video"
if not os.path.exists(medvid_folder):
    os.makedirs(medvid_folder)

eval_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/evaluation/medivdqa"
if not os.path.exists(eval_folder):
    os.makedirs(eval_folder)

vid_json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/medvidqa"
if not os.path.exists(vid_json_folder):
    os.makedirs(vid_json_folder)

In [11]:
redownload_list = ['21s9tRvo_x4.mp4', '3C01Cqd1mHs.mp4', '1vnjCOq4rGQ.mp4', 'Qaez8OKf-eE.mp4', 'd8-GG4KD_Tw.mp4', 'LxwjHzTpXV4.mp4', 'wEfiJL-JS_4.mp4', 'U4u6jyJjDT4.mp4']

In [44]:
list_of_bad_videos = ['SztsZNp-jDM', 'OKXoHwkx55c', 'V9j5JkWGwI8', 'ehl2MPczYoQ', 'mMNloo140pU', 'QwhD5UTUW60', 'Ehan_VI7p4c']

In [36]:
train_path = '/home/hlpark/shared/MedVidQA/train.json'
val_path = '/home/hlpark/shared/MedVidQA/val.json'
test_path = '/home/hlpark/shared/MedVidQA/test.json'
txt_file_root = "/home/hlpark/shared/MedVidQA"

In [37]:
train = json.load(open(train_path))
val = json.load(open(val_path))
test = json.load(open(test_path))

In [38]:
new_train = {}
new_val = {}
new_test = {}
video_names = {}
vid_dict_train, vid_dict_val, vid_dict_test = {}, {}, {}
unav_train_query_cnt, unav_val_query_cnt, unav_test_query_cnt, total_test_cnt, total_val_cnt = 0, 0, 0, 0, 0

with open(os.path.join(txt_file_root, "medvidqa_train_queries.txt"), "w") as f:
    for i, qa in tqdm(enumerate(train)):
        if "aNct84-0WZk" in qa["video_id"]:
            #this video doesn't have audio and causes minilm text embedding extraction error due to empty string
            #will not be used for hirest training set 
            continue
        if qa['video_id'] in list_of_bad_videos:
            continue
        qa_dict = {}
        video_id = qa['video_id'] + ".mp4"
        qa_dict[video_id] = {}
        qa_dict[video_id]['relevant'] = True
        qa_dict[video_id]['clip'] = True
        qa_dict[video_id]['v_duration'] = qa['video_length']
        qa_dict[video_id]['bounds'] = [qa['answer_start_second'], qa['answer_end_second']]
        qa_dict[video_id]['steps'] = []
        qa_dict['qid'] = qa['sample_id']
        f.writelines(qa['question'] + "\n")
        total_test_cnt += 1
        if os.path.exists(os.path.join(medvid_folder, qa['video_id'] + ".mp4")):
            new_train[qa['question']] = qa_dict
            #only append when video is available for downloading
            if qa['video_id'] not in vid_dict_train:
                vid_dict_train[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
            continue
        vid = qa['video_id']
        dl_ops = {
        'outtmpl': f'{medvid_folder}/{vid}.mp4'
        }
        try:
            with YoutubeDL(dl_ops) as ydl:
                ydl.download(qa['video_url'])
            new_train.append(qa_dict)
            #only append when video is available for downloading
            if qa['video_id'] not in vid_dict_train:
                vid_dict_train[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
            
        except:
            print("Unvailable ", qa['video_id'])
            unav_train_query_cnt += 1

with open(os.path.join(txt_file_root, "medvidqa_test_queries.txt"), "w") as f:
    for i, qa in enumerate(test):
        qa_dict = {}
        video_id = qa['video_id'] + ".mp4"
        qa_dict[video_id] = {}
        qa_dict[video_id]['relevant'] = True
        qa_dict[video_id]['clip'] = True
        qa_dict[video_id]['v_duration'] = qa['video_length']
        qa_dict[video_id]['bounds'] = [qa['answer_start_second'], qa['answer_end_second']]
        qa_dict[video_id]['steps'] = []
        qa_dict['qid'] = qa['sample_id']
        f.writelines(qa['question'] + "\n")
        total_test_cnt += 1
        if os.path.exists(os.path.join(medvid_folder, qa['video_id'] + ".mp4")):
            new_test[qa['question']] = qa_dict
            #only append when video is available for downloading
            if qa['video_id'] not in vid_dict_test:
                vid_dict_test[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
            continue
        vid = qa['video_id']
        dl_ops = {
        'outtmpl': f'{medvid_folder}/{vid}.mp4'
        }
        try:
            with YoutubeDL(dl_ops) as ydl:
                ydl.download(qa['video_url'])
            new_test.append(qa_dict)
            #only append when video is available for downloading
            if qa['video_id'] not in vid_dict_test:
                vid_dict_test[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
            
        except:
            print("Unvailable ", qa['video_id'])
            unav_test_query_cnt += 1

with open(os.path.join(txt_file_root, "medvidqa_val_queries.txt"), "w") as f:
    for i, qa in enumerate(val):
        qa_dict = {}
        video_id = qa['video_id'] + ".mp4"
        qa_dict[video_id] = {}
        qa_dict[video_id]['relevant'] = True
        qa_dict[video_id]['clip'] = True
        qa_dict[video_id]['v_duration'] = qa['video_length']
        qa_dict[video_id]['bounds'] = [qa['answer_start_second'], qa['answer_end_second']]
        qa_dict[video_id]['steps'] = []
        qa_dict['qid'] = qa['sample_id']
        f.writelines(qa['question'] + "\n")
        total_val_cnt += 1
        if os.path.exists(os.path.join(medvid_folder, qa['video_id'] + ".mp4")):
            new_val[qa['question']] = qa_dict
            if qa['video_id'] not in vid_dict_val:
                vid_dict_val[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
            continue
        vid = qa['video_id']
        dl_ops = {
        'outtmpl': f'{medvid_folder}/{vid}.mp4'
        }
        try:
            with YoutubeDL(dl_ops) as ydl:
                ydl.download(qa['video_url'])
            new_val.append(qa_dict)
            #only append when video is available for downloading
            if qa['video_id'] not in vid_dict_val:
                vid_dict_val[video_id] = qa_dict[video_id]['v_duration']
            if qa['video_id'] not in video_names:
                video_names[qa['video_id']] = []
        except:
            print("Unvailable ", qa['video_id'])
            unav_val_query_cnt += 1
print(f"{unav_val_query_cnt}/ {total_val_cnt}, {unav_test_query_cnt} / {total_test_cnt} ")

0it [00:00, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=zWG4LWPraYw
[youtube] zWG4LWPraYw: Downloading webpage
[youtube] zWG4LWPraYw: Downloading ios player API JSON
[youtube] zWG4LWPraYw: Downloading android player API JSON


ERROR: [youtube] zWG4LWPraYw: Video unavailable
635it [00:05, 112.60it/s]

Unvailable  zWG4LWPraYw
[youtube] Extracting URL: https://www.youtube.com/watch?v=WU9-MmRe4_g
[youtube] WU9-MmRe4_g: Downloading webpage
[youtube] WU9-MmRe4_g: Downloading ios player API JSON
[youtube] WU9-MmRe4_g: Downloading android player API JSON


ERROR: [youtube] WU9-MmRe4_g: Private video. Sign in if you've been granted access to this video
902it [00:06, 154.27it/s]

...

Unvailable  0YsX8qv0FaI
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZO_G0-jBVH8
[youtube] ZO_G0-jBVH8: Downloading webpage
[youtube] ZO_G0-jBVH8: Downloading ios player API JSON
[youtube] ZO_G0-jBVH8: Downloading android player API JSON


ERROR: [youtube] ZO_G0-jBVH8: Video unavailable


Unvailable  ZO_G0-jBVH8
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZO_G0-jBVH8
[youtube] ZO_G0-jBVH8: Downloading webpage
[youtube] ZO_G0-jBVH8: Downloading ios player API JSON
[youtube] ZO_G0-jBVH8: Downloading android player API JSON


ERROR: [youtube] ZO_G0-jBVH8: Video unavailable


Unvailable  ZO_G0-jBVH8
4/ 145, 7 / 2844 


In [39]:
save_json(new_train, f'{vid_json_folder}/all_data_train.json')
save_json(new_val, f'{vid_json_folder}/all_data_val.json')
save_json(new_test, f'{vid_json_folder}/all_data_test.json')

In [50]:
#dont need to run it, this is for verifying vid length and frame extraction
raw_frames_path = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/medvidqa/raw_frames"
for r, d, f in os.walk(raw_frames_path):
    for dir in d:
        for _, _, files in os.walk(os.path.join(r, dir)):
            if dir in vid_dict_val and vid_dict_val[dir]!= len(files):
                print(dir, len(files), vid_dict_val[dir])
            elif dir in vid_dict_test and vid_dict_test[dir] != len(files):
                print(dir, len(files), vid_dict_test[dir])
            else:
                if dir not in vid_dict_val and dir not in vid_dict_test:
                    print("not in any")

O1kiguGUt3o.mp4 98 97
5GO18A5-ZtQ.mp4 573 572
CvW-Zq3NlkU.mp4 84 83
wkF8OwsyNNQ.mp4 118 117
nopgqBoUmYg.mp4 103 102
71LlfdW548U.mp4 312 311
gJOMV2mZ1B0.mp4 72 71
E92qqAftUak.mp4 78 77
s4wUKmr2xtA.mp4 827 826
pYzUQb79_Rw.mp4 326 325
tr7JKUxwqwA.mp4 303 302
2oXoiQfija4.mp4 408 407
AtKU8zw2Jxg.mp4 596 595
N59oOxyaE1A.mp4 618 617
RRLY6anXXQo.mp4 71 70
lwhYbWFtNuE.mp4 137 136
hJ3XICFLxvU.mp4 361 360
lfhDPB8LaTk.mp4 392 391
bUgV89cdfTM.mp4 105 104
6kQEDRQdJZ8.mp4 583 582
kSpggqOLgaU.mp4 451 450
0dr5yuoBOF4.mp4 377 376
M6WV95X0fRk.mp4 91 90
fsdchGejKmU.mp4 618 617
g-gNQPyxU4c.mp4 686 685
UZqktEPlxTU.mp4 119 118
LL854--GBy4.mp4 586 585
VykLxY9mzug.mp4 742 741
GUg4zMpchhk.mp4 530 529
1Nr6wPFz09A.mp4 227 226
B9yiDTONlOs.mp4 157 156
hekjiCqb9-g.mp4 105 104
-Zx3pWhMBjw.mp4 234 233
4h3V6F4Rl_k.mp4 406 405
vgW6ZD_QwwQ.mp4 411 410
CWRWeoaqi8k.mp4 423 422
R9oWYkK_l3M.mp4 165 164
xYFeFFzFftw.mp4 566 565
not in any
not in any
not in any
not in any
not in any
not in any
not in any
not in any
not in any
n

In [92]:
weak_label_path = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/medvidqa/five_labeled_pred_med_from_gt_vid_dict.json"
weak_label = json.load(open(weak_label_path))

In [93]:
list_of_bad_videos = ['SztsZNp-jDM', 'OKXoHwkx55c', 'V9j5JkWGwI8', 'ehl2MPczYoQ', 'mMNloo140pU', 'QwhD5UTUW60', 'Ehan_VI7p4c']

In [106]:
error_videos = ['G2tP1mJHbAg', 'lUBQ7-uOWKw', 'GSbjwzyuUoo', 'f_w82yx87KI', 'ILoZc6rpdU8', 'Q8m25MkEr0o', 'EnAyXA3_Mo4', 'Ur7UCjQTV9E', '6_hrMBnA1VA', 'Mx0PPNwfdvE', 'Q1ifxZ3lR2o']

In [107]:
new_train_med = {}
new_train_nonmed = {}
new_train_both = {}
for i, qa in tqdm(enumerate(train)):
    if "aNct84-0WZk" in qa["video_id"]:
        #this video doesn't have audio and causes minilm text embedding extraction error due to empty string
        #will not be used for hirest training set 
        continue
    if qa['video_id'] in list_of_bad_videos + error_videos:
        continue
    qa_dict = {}
    video_id = qa['video_id'] + ".mp4"
    qa_dict[video_id] = {}
    qa_dict[video_id]['relevant'] = True
    qa_dict[video_id]['clip'] = True
    qa_dict[video_id]['v_duration'] = qa['video_length']
    qa_dict[video_id]['bounds'] = [qa['answer_start_second'], qa['answer_end_second']]
    qa_dict[video_id]['steps'] = []
    qa_dict['qid'] = qa['sample_id']
    
    if qa['video_id'] in weak_label:
        for idx, q in enumerate(weak_label[qa['video_id']]):
            if qa['question'] not in q:
                continue
            if q[qa['question']] == "med":
                new_train_med[qa['question']] = qa_dict
            elif q[qa['question']] == "nonmed":
                new_train_nonmed[qa['question']] = qa_dict
            new_train_both[qa['question']] = qa_dict
        


2710it [00:00, 74896.81it/s]


In [108]:
save_json(new_train_med, f'{vid_json_folder}/visual_medical_train.json')
save_json(new_train_nonmed, f'{vid_json_folder}/visual_nonmedical_train.json')
save_json(new_train_both, f'{vid_json_folder}/postprocessed_train.json')